<a href="https://colab.research.google.com/github/hyeonji0401/CUDA_practice/blob/main/CUDA_sharedMemory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 9.1 공유 메모리 사용 방법
- 크게 다음 3가지로 구분할 수 있음
  - 스레드 블록 내 스레드들의 공유 데이터 보관
  - L1 캐시(HW 관리 캐시)
  - 사용자 관리(user-managed) 캐시

1.스레드 블록 내 스레드들의 공유 데이터 보관
- 만약 블록 내 스레드들 사이에 정보 또는 데이터를 주고 받아야한다면 공유 메모리에 변수 등 공간을 만들고 여러 스레드들이 해당 공간에 데이터를 읽고 쓰는 형태로 구현 가능함
- 동기화를 주의해야함
  - 공유 메모리는 여러 스레드가 동시에 데이터를 쓰거나 읽을 수 있기 때문에 읽고 쓰는 순서에 대한 약속을 지켜야함
  - 동기화란 스레드들 사이에 순서를 맞추는 행위 또는 서로 정보를 교환하는 행위라고 말할 수 있음
  - 볼록 내 스레드들에 대한 동기화함수는 _syncthreads()
    - 블록 내 모든 스레드가 해당 지점에 도착할 때까지 다른 스레드들의 진행을 막는 역할을 함

2.L1캐시(HW 관리 캐시)
- 스레드들이 공유하는 데이터를 판단하기 어렵고, 자주 사용하는 데이터에 대한 분류 및 관리를 직접 하기 어려운 경우 간단히 사용할 수 있는 방법
- 공유 메모리를 L1 캐시로 사용하면 일반적인 캐시와 유사한 형태로 동작함

**캐시의 동작원리**

> - 캐시는 일반적으로 코어 내부에 있는 메모리를 말함
- 시스템 메모리에 대해 입출력 병목 현상을 해소하는 역할을 함
- 크기가 매우 작지만 캐시가 입출력 병목 현상에 도움이 되는 이유는 지역성 때문
 - 대표적 예시는 for문

- GPU의 캐시 또한 CPU 캐시와 유사하게 동작함
- 그러나 GPU는 SIMT의 구조를 가지므로 32개의 스레드가 동시에 접근함
  - 따라서 워프 내 스레드들의 메모리 접근 패턴에 따라 성능이 크게 달라질 수 있음
- 워프 내 스레드들이 한 번에 접근하는 데이터 블록의 수가 메모리 접근 횟수가 됨
  - 이상적인 경우 : 1번 / 최악의 경우 : 32번
- 따라서 워프 내 스레드들이 동일한 또는 인접한 데이터에 접근하도록 스레드 레이아웃 및 알고리즘을 설계하면 그렇지 않은 경우 대비 큰 성능 향상을 기대할 수 있음

3.사용자 관리 캐시
- L1캐시는 HW가 관리하기 때문에 알고리즘의 특징을 반영한 최적의 캐시 활용 효율이 얻기 어려움
- 사용자 관리 캐시란 알고리즘의 특성을 기반으로 데이터 접근 패턴을 파악하고 그에 따라 사용자가 직접 목표 데이터를 공유 메모리 영역으로 가져오거나 내리면서 공유 메모리는 캐시처럼 사용하는 것을 말함